In [2]:
import pandas as pd
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
import xgboost as xgb 
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715212021978

In [4]:
mlflow.set_tracking_uri("file:./mlruns")  # Ensure logging to local directory

mlflow.set_experiment("xgboost_2")  # Optional: organize your runs



<Experiment: artifact_location=('file:d:/Github '
 'repos/mlops-zoomcamp/02-experiment_tracking/mlruns/482108769645636610'), creation_time=1747986607300, experiment_id='482108769645636610', last_update_time=1747986607300, lifecycle_stage='active', name='xgboost_2', tags={}>

In [5]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)


In [6]:
import mlflow.xgboost


with mlflow.start_run():

    best_params = {
        'max_depth': 6,
        'learning_rate': 0.1,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1,
        'min_child_weight': 1,
        'objective': 'reg:linear',
        'seed': 42,
    }
    mlflow.log_params(best_params)
    booster = xgb.train(
        params = best_params,
        dtrain = train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50,
        verbose_eval=False   
            ) 
        
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_param("booster", booster)    
    mlflow.xgboost.save_model(booster, path="models_mlflow")


c:\Users\Ultracell\anaconda3\envs\MLOPs\Lib\site-packages\xgboost\callback.py:386: UserWarning: [08:58:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


MlflowException: Path 'd:\Github_repos\mlops-zoomcamp\02-experiment_tracking\models_mlflow' already exists and is not empty